In [1]:
!pip install -U textacy==0.6.3

Requirement already up-to-date: textacy==0.6.3 in /usr/local/lib/python3.6/dist-packages (0.6.3)


In [2]:
import textacy
import spacy
nlp = spacy.load('en')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from collections import Counter
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import os
import random
import codecs

random.seed(42)

stemmer = PorterStemmer()

import numpy as np

np.random.seed(42)

def train(classifier, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

    classifier.fit(X_train, y_train)
    print("Accuracy: %s" % classifier.score(X_test, y_test))
    return classifier

### Download dataset from
http://myleott.com/op-spam.html

and uncompress it in some folder

In [19]:
!mkdir '/tmp/spam'
!wget -P '/tmp/spam' 'https://myleott.com/op_spam_v1.4.zip'
!mkdir '/tmp/spam/op_spam_v1.4'
!unzip -o -q '/tmp/spam/op_spam_v1.4.zip' -d '/tmp/spam/op_spam_v1.4'


mkdir: cannot create directory ‘/tmp/spam’: File exists
--2019-11-07 15:28:42--  https://myleott.com/op_spam_v1.4.zip
Resolving myleott.com (myleott.com)... 185.199.109.153, 185.199.108.153
Connecting to myleott.com (myleott.com)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1172634 (1.1M) [application/zip]
Saving to: ‘/tmp/spam/op_spam_v1.4.zip.8’

op_spam_v1.4.zip.8  100%[===================>]   1.12M  --.-KB/s    in 0.08s   

2019-11-07 15:28:42 (14.7 MB/s) - ‘/tmp/spam/op_spam_v1.4.zip.8’ saved [1172634/1172634]

mkdir: cannot create directory ‘/tmp/spam/op_spam_v1.4’: File exists


In [0]:
base_path = "/tmp/spam/op_spam_v1.4"

In [0]:
def read_review(filename):
    with open(filename, "rb") as fp:
        return fp.read()

In [22]:
real_reviews = []
fake_reviews = []

for dir_name, subdir_list, file_list in os.walk(base_path):
    if "deceptive" in dir_name:
        for fname in file_list:
            if ".txt" in fname:
                fake_reviews.append(read_review(os.path.join(dir_name,fname)))
    if "truthful" in dir_name:
        for fname in file_list:
            if ".txt" in fname:
                real_reviews.append(read_review(os.path.join(dir_name,fname)))
          
print(f"{len(fake_reviews)} fake reviews, {len(real_reviews)} real reviews")

800 fake reviews, 800 real reviews


In [23]:
indexes = list(range(len(fake_reviews)+len(real_reviews)))
random.shuffle(indexes)
indexes[:10]

[856, 549, 970, 111, 736, 517, 586, 1326, 808, 491]

In [24]:
real_reviews[0]

b'My wife and I redeemed some of my Hilton Reward Points to stay a few nights at the Palmer House. The hotel is absolutely beautiful. From the moment you walk into the lobby you can tell that this place is nice. As we were Diamond members we got upgraded to a suite which was great because we were provided access to their executive lounge (free food & drinks). The service was excellent and the location of the hotel is within walking distance of many sights on the south side of Chicago. I would definitely recommend this hotel to anyone interested in coming to Chicago. \n'

In [0]:
data = []
target = []
all_reviews = fake_reviews + real_reviews
for i in indexes:
    data.append(all_reviews[i])
    if i <= len(fake_reviews):
        target.append(0)
    else:
        target.append(1)
    

In [26]:
target[:10]

[1, 0, 1, 0, 0, 0, 0, 1, 1, 0]

In [27]:
%%time

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
 
trial1 = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', BernoulliNB()),
])
 
train(trial1, data, target)
# Accuracy: 0.8463497453310697


Accuracy: 0.8575
CPU times: user 213 ms, sys: 3.37 ms, total: 216 ms
Wall time: 249 ms


In [28]:
%%time

from sklearn.svm import SVC

C = 1.0  # SVM regularization parameter

trial_svc = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SVC(kernel='linear', C=C)),
])
 
train(trial_svc, data, target)
# Accuracy: 0.8463497453310697


Accuracy: 0.86
CPU times: user 1.41 s, sys: 11.1 ms, total: 1.42 s
Wall time: 1.43 s


In [29]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
from nltk.corpus import stopwords


trial2 = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', MultinomialNB()),
])
 
train(trial2, data, target)
# Accuracy: 0.88


Accuracy: 0.855


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                   

In [31]:
trial3 = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', BernoulliNB()),
])
 
train(trial3, data, target)
# Accuracy: 0.88


Accuracy: 0.855


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
              

In [32]:
from nltk.corpus import stopwords


trial2 = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', MultinomialNB(alpha=0.05)),
])
 
train(trial2, data, target)
# Accuracy: 0.875


Accuracy: 0.82


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                   

In [33]:
trial3 = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', MultinomialNB(alpha=2)),
])
 
train(trial3, data, target)


Accuracy: 0.86


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                   

In [0]:
import textacy

In [0]:
from textacy.preprocess import preprocess_text

In [37]:
preprocess_text(str(data[0]), lowercase=True, no_punct=True, no_urls=True)

'b after reading some of the most recent reviews i m really perplexed i didn t know what to expect but i was most pleasantly surprised just as all the past reviews both negative and positive have stated you can t beat the location of this hotel it s perfect we had a big spacious room in the old tower we avoided the expensive pitfalls of the hotel extras such as bottled water and items in the room fridge the beds were great the rooms were kept clean and the staff was very friendly great martinis in the hotel bar i really can t understand the negative reviews i know you can t please everyone but in some cases i simply don t believe their claims i ve traveled a lot and have stayed in plenty of hotels this is a great hotel n'

In [0]:
cleaned_data = [preprocess_text(str(text), no_emails=True, no_punct=True, no_urls=True, lowercase=True) for text in data]

In [39]:
trial_c = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', MultinomialNB(alpha=1)),
])

train(trial_c, cleaned_data, target)

Accuracy: 0.855


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                   

In [40]:
from sklearn.linear_model import SGDClassifier

trial_l = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('classifier', SGDClassifier()),
])

train(trial_c, data, target)

Accuracy: 0.855


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves'...
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                   

In [41]:
trial1.predict([data[0]])

array([1])

In [42]:
trial_c.predict([data[0]])

array([1])

In [43]:
data[0]

b"After reading some of the most recent reviews I'm really perplexed. I didn't know what to expect, but I was most pleasantly surprised. Just as all the past reviews, both negative and positive, have stated, you can't beat the location of this hotel. It's perfect. We had a big spacious room in the old tower. We avoided the expensive pitfalls of the hotel extras such as bottled water and items in the room fridge. The beds were great, the rooms were kept clean and the staff was very friendly. Great martinis in the hotel bar. I really can't understand the negative reviews. I know you can't please everyone, but in some cases I simply don't believe their claims. I've traveled a lot and have stayed in plenty of hotels. This is a great hotel! \n"

In [44]:
target[0]

1

In [0]:
from sklearn.metrics import classification_report

In [46]:
%%time 

results_c = trial_c.predict(data)

CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 170 ms


In [47]:
print(classification_report(target, results_c))

              precision    recall  f1-score   support

           0       0.92      0.98      0.94       801
           1       0.97      0.91      0.94       799

    accuracy                           0.94      1600
   macro avg       0.94      0.94      0.94      1600
weighted avg       0.94      0.94      0.94      1600



In [48]:
for i in range(10):
    print(f"review {i}")
    print(data[i])

print("true or false?")
print("...")
for i in range(10):
    if target[i]:
        print(f"review {i}", True)
    else:
        print(f"review {i}", False)

review 0
b"After reading some of the most recent reviews I'm really perplexed. I didn't know what to expect, but I was most pleasantly surprised. Just as all the past reviews, both negative and positive, have stated, you can't beat the location of this hotel. It's perfect. We had a big spacious room in the old tower. We avoided the expensive pitfalls of the hotel extras such as bottled water and items in the room fridge. The beds were great, the rooms were kept clean and the staff was very friendly. Great martinis in the hotel bar. I really can't understand the negative reviews. I know you can't please everyone, but in some cases I simply don't believe their claims. I've traveled a lot and have stayed in plenty of hotels. This is a great hotel! \n"
review 1
b"The Swissotel Chicago hotel aspires to be a tourist's paradise, a hotel so grand and luxurious that you'd rather stay than return home at the end of your trip. But is it really as magnificent as suggested? True, you can get a mass

In [49]:
misclassified =  []
for i, item in enumerate(target):
    if item != results_c[i]:
        misclassified.append((data[i], item))
len(misclassified)

91

In [50]:
misclassified[:10]

[(b"This hotel had a great location, but you can do much better for the money. The hallways smelled of smoke, dirty elevators, furniture in the lobby was thread bare. The staff was very helpful. The room was large, bed comfortable but the room was so musty and old smoke lingered. If you have any allergies don't stay here!\n",
  1),
 (b"My fiance and I traveled to Chicago for the first time this December and we were not happy with this hotel. The friendliest people on the staff were the bell boys. They greeted us with a smile and gave us lots of great advice throughout our stay. The rest of the staff however was terrible. When we first checked in, I was instantly discouraged. The front desk person was not in a good mood and I couldn't wait to leave the lobby. Having not traveled here before we were looking for exciting places to eat and to go sightseeing. We asked multiple staff members at the front desk of places to go and see and never got any good recommendations. We felt as if we we

In [51]:
from sklearn.metrics import confusion_matrix

confusion_matrix(target, results_c)

array([[781,  20],
       [ 71, 728]])

In [52]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [53]:
%%time

import string 

def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]
 
trial_stem = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=stemming_tokenizer,
                             stop_words=stopwords.words('english') + list(string.punctuation))),
    ('classifier', MultinomialNB(alpha=1)),
])

train(trial_stem, data, target)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Accuracy: 0.86
CPU times: user 6.26 s, sys: 5.56 ms, total: 6.26 s
Wall time: 6.28 s


In [0]:
def extract_words(text):
    lemmas = [word.lemma_ for word in nlp(text) if not word.is_stop]
    return lemmas
    
fake_words = []

In [0]:
text= data[0]

In [56]:
text

b"After reading some of the most recent reviews I'm really perplexed. I didn't know what to expect, but I was most pleasantly surprised. Just as all the past reviews, both negative and positive, have stated, you can't beat the location of this hotel. It's perfect. We had a big spacious room in the old tower. We avoided the expensive pitfalls of the hotel extras such as bottled water and items in the room fridge. The beds were great, the rooms were kept clean and the staff was very friendly. Great martinis in the hotel bar. I really can't understand the negative reviews. I know you can't please everyone, but in some cases I simply don't believe their claims. I've traveled a lot and have stayed in plenty of hotels. This is a great hotel! \n"

In [57]:
extract_words(str(text))

['b"after',
 'read',
 'recent',
 'review',
 'perplexed',
 '.',
 'know',
 'expect',
 ',',
 'pleasantly',
 'surprised',
 '.',
 'past',
 'review',
 ',',
 'negative',
 'positive',
 ',',
 'state',
 ',',
 'beat',
 'location',
 'hotel',
 '.',
 'perfect',
 '.',
 'big',
 'spacious',
 'room',
 'old',
 'tower',
 '.',
 'avoid',
 'expensive',
 'pitfall',
 'hotel',
 'extra',
 'bottled',
 'water',
 'item',
 'room',
 'fridge',
 '.',
 'bed',
 'great',
 ',',
 'room',
 'keep',
 'clean',
 'staff',
 'friendly',
 '.',
 'great',
 'martini',
 'hotel',
 'bar',
 '.',
 'understand',
 'negative',
 'review',
 '.',
 'know',
 ',',
 'case',
 'simply',
 'believe',
 'claim',
 '.',
 'travel',
 'lot',
 'stay',
 'plenty',
 'hotel',
 '.',
 'great',
 'hotel',
 '!',
 '\\n',
 '"']

In [0]:
cleaned_fake = [textacy.preprocess_text(str(text), no_emails=True, no_punct=True, no_urls=True, lowercase=True, no_numbers=True) for text in fake_reviews]


In [0]:
cleaned_real = [textacy.preprocess_text(str(text), no_emails=True, no_punct=True, no_urls=True, lowercase=True, no_numbers=True) for text in real_reviews]


In [60]:
extract_words(cleaned_fake[0])

['b',
 'extravagant',
 'exuberant',
 'experience',
 'stay',
 'hotel',
 'allegro',
 'enjoyable',
 'intoxicating',
 'view',
 'away',
 'relaxing',
 'exceptional',
 'room',
 'service',
 'room',
 'decorative',
 'par',
 'enjoy',
 'complimentary',
 'tea',
 'highly',
 'recommend',
 'hotel',
 'allegro',
 'stay',
 'vacation',
 'away',
 'n']

In [0]:
real_words = Counter([word for line in cleaned_real for word in extract_words(line)])

In [0]:
fake_words = Counter([word for line in cleaned_fake for word in extract_words(line)])


In [63]:
real_words.most_common(20)

[('room', 1747),
 ('hotel', 1658),
 ('number', 1433),
 ('stay', 1052),
 ('n', 808),
 ('b', 806),
 ('t', 575),
 ('great', 554),
 ('chicago', 487),
 ('night', 469),
 ('staff', 447),
 ('service', 410),
 ('bed', 380),
 ('good', 359),
 ('location', 358),
 ('s', 346),
 ('$', 316),
 ('check', 310),
 ('nice', 301),
 ('time', 285)]

In [64]:
fake_words.most_common(20)

[('hotel', 1987),
 ('room', 1756),
 ('stay', 1208),
 ('chicago', 1040),
 ('b', 804),
 ('n', 800),
 ('t', 544),
 ('number', 508),
 ('service', 464),
 ('staff', 403),
 ('like', 355),
 ('time', 343),
 ('great', 320),
 ('check', 306),
 ('night', 297),
 ('look', 296),
 ('s', 294),
 ('clean', 278),
 ('bed', 277),
 ('good', 273)]

In [0]:
fake_set = set([word for word, _ in fake_words.most_common(50)])

In [0]:
real_set = set([word for word, _ in real_words.most_common(50)])

In [67]:
fake_set - real_set

{'arrive',
 'business',
 'city',
 'definitely',
 'expect',
 'experience',
 'feel',
 'food',
 'hour',
 'need',
 'recommend',
 'take',
 'visit',
 'wait',
 'want'}

In [68]:
real_set - fake_set

{'$',
 'bar',
 'bathroom',
 'book',
 'breakfast',
 'call',
 'comfortable',
 'door',
 'floor',
 'lobby',
 'location',
 'pay',
 'small',
 'tell',
 'walk'}

In [69]:
fake_set & real_set

{'area',
 'ask',
 'b',
 'bed',
 'check',
 'chicago',
 'clean',
 'come',
 'day',
 'desk',
 'didn',
 'find',
 'friendly',
 'get',
 'go',
 'good',
 'great',
 'hotel',
 'like',
 'look',
 'n',
 'nice',
 'night',
 'number',
 'place',
 'price',
 'restaurant',
 'room',
 's',
 'service',
 'staff',
 'stay',
 't',
 'time',
 'view'}